In [1]:
import pandas as pd
from glob import glob
import pyranges as pr

In [2]:
# Import annotation
gtf = pr.read_gtf('../../data/genomes/GRCm38/gencode.vM22.primary_assembly.annotation.gtf', as_df=True)
gtf = gtf.loc[gtf.Feature == 'transcript']

In [3]:
# Import quantfiles
quantfiles = sorted(glob('../../data/MihaDeseq/salmon_quantfiles/*quant.sf'))

# Save to
outpath = '../../data/MihaDeseq'

In [4]:
TranscriptIds = gtf[['gene_id', 'transcript_id']]
TranscriptIds['stable_gene_id'] = TranscriptIds.gene_id.apply(lambda x: x.split('.')[0])
TranscriptIds['transcript_stable_id'] = TranscriptIds.transcript_id.apply(lambda x: x.split('.')[0])
TranscriptIds.head()

/home/klara/anaconda3/envs/JupyterAnalyses/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/klara/anaconda3/envs/JupyterAnalyses/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,gene_id,transcript_id,stable_gene_id,transcript_stable_id
1,ENSMUSG00000102693.1,ENSMUST00000193812.1,ENSMUSG00000102693,ENSMUST00000193812
4,ENSMUSG00000064842.1,ENSMUST00000082908.1,ENSMUSG00000064842,ENSMUST00000082908
7,ENSMUSG00000051951.5,ENSMUST00000162897.1,ENSMUSG00000051951,ENSMUST00000162897
10,ENSMUSG00000051951.5,ENSMUST00000159265.1,ENSMUSG00000051951,ENSMUST00000159265
13,ENSMUSG00000051951.5,ENSMUST00000070533.4,ENSMUSG00000051951,ENSMUST00000070533


In [5]:
TranscriptIds[['transcript_stable_id', 'stable_gene_id']].to_csv(f'{outpath}/transcript_to_gene_mapping_mm10_V22.csv', index=False)

In [6]:
DfCountsTPM = pd.DataFrame()
for i, f in enumerate(quantfiles):
    s = f.split('/')[-1].split('.')[0]
    df = pd.read_csv(f, sep='\t', index_col=0).rename(columns={'TPM': f'{s}.TPM'})
    df = df[f'{s}.TPM']
    # print(df.head())
    DfCountsTPM[f'{s}.TPM'] = df
# Convert transcript counts to gene counts
DfCountsTPM = DfCountsTPM.merge(TranscriptIds[['transcript_id', 'stable_gene_id']], left_on='Name', right_on='transcript_id', how='inner').set_index('transcript_id')
# Aggregate counts for the same gene_id
dfGeneCountsTPM = DfCountsTPM.groupby('stable_gene_id').sum().astype(float)
dfGeneCountsTPM.to_csv(f'{outpath}/GeneLevel_TPM_counts.csv')